In [1]:
import os
import keras
os.environ["CUDA_VISIBLE_DEVICES"] = "" # 使用 CPU

import numpy as np
import cv2 # 載入 cv2 套件
import matplotlib.pyplot as plt

train, test = keras.datasets.cifar10.load_data()

Using TensorFlow backend.


In [2]:
x_train, y_train = train
x_test, y_test = test
y_train = y_train.astype(int)
y_test = y_test.astype(int)

## 產生直方圖特徵的訓練資料

In [3]:
x_train_histogram = []
x_test_histogram = []

# 對於所有訓練資料
for i in range(len(x_train)):
    chans = cv2.split(x_train[i]) # 把圖像的 3 個 channel 切分出來
    # 對於所有 channel
    hist_feature = []
    for chan in chans:
        # 計算該 channel 的直方圖
        hist = cv2.calcHist([chan], [0], None, [16], [0, 256]) # 切成 16 個 bin
        hist_feature.extend(hist.flatten())
    # 把計算的直方圖特徵收集起來
    x_train_histogram.append(hist_feature)

# 對於所有測試資料也做一樣的處理
for i in range(len(x_test)):
    chans = cv2.split(x_test[i]) # 把圖像的 3 個 channel 切分出來
    # 對於所有 channel
    hist_feature = []
    for chan in chans:
        # 計算該 channel 的直方圖
        hist = cv2.calcHist([chan], [0], None, [16], [0, 256]) # 切成 16 個 bin
        hist_feature.extend(hist.flatten())
    x_test_histogram.append(hist_feature)

x_train_histogram = np.array(x_train_histogram)
x_test_histogram = np.array(x_test_histogram)

## 產生 HOG 特徵的訓練資料

HOG 特徵通過計算和統計圖像局部區域的梯度方向直方圖來構建特徵，具體細節不在我們涵蓋的範圍裡面，有興趣的同學請參考[補充資料](https://www.cnblogs.com/zyly/p/9651261.html)哦

In [4]:
# SZ=20
bin_n = 16 # Number of bins

def hog(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)
    bins = np.int32(bin_n*ang/(2*np.pi))    # quantizing binvalues in (0...16)
    bin_cells = bins[:10,:10], bins[10:,:10], bins[:10,10:], bins[10:,10:]
    mag_cells = mag[:10,:10], mag[10:,:10], mag[:10,10:], mag[10:,10:]
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)     # hist is a 64 bit vector
    return hist.astype(np.float32)

x_train_hog = np.array([hog(x) for x in x_train])
x_test_hog = np.array([hog(x) for x in x_test])

## SVM model

SVM 是機器學習中一個經典的分類算法，具體細節有興趣可以參考 [知乎上的解釋](https://www.zhihu.com/question/21094489)，我們這裡直接調用 opencv 中實現好的函數

## 用 histogram 特徵訓練 SVM 模型

訓練過程可能會花點時間，請等他一下

In [5]:
SVM_hist = cv2.ml.SVM_create()
SVM_hist.setKernel(cv2.ml.SVM_LINEAR)
SVM_hist.setGamma(5.383)
SVM_hist.setType(cv2.ml.SVM_C_SVC)
SVM_hist.setC(2.67)

#training
SVM_hist.train(x_train_histogram, cv2.ml.ROW_SAMPLE, y_train)

# prediction
_, y_hist_train = SVM_hist.predict(x_train_histogram)
_, y_hist_test = SVM_hist.predict(x_test_histogram)

## 用 HOG 特徵訓練 SVM 模型

訓練過程可能會花點時間，請等他一下

In [6]:
SVM_hog = cv2.ml.SVM_create()
SVM_hog.setKernel(cv2.ml.SVM_LINEAR)
SVM_hog.setGamma(5.383)
SVM_hog.setType(cv2.ml.SVM_C_SVC)
SVM_hog.setC(2.67)

#training
SVM_hog.train(x_train_hog, cv2.ml.ROW_SAMPLE, y_train)

# prediction
_, y_hog_train = SVM_hog.predict(x_train_hog)
_, y_hog_test = SVM_hog.predict(x_test_hog)

## 作業

嘗試比較用 color histogram 和 HOG 特徵來訓練的 SVM 分類器在 cifar10 training 和 testing data 上準確度的差別

In [7]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_hist_test))
print(accuracy_score(y_test, y_hog_test))

0.1444
0.2079


In [8]:
print('For HIST:')
print('Accuracy of training data: %.4f'%(y_hist_train==y_train).mean())
print('Accuracy of testing data: %.4f'%(y_hist_test==y_test).mean())

print("\n")

print('For HOG:')
print('Accuracy of training data: %.4f'%(y_hog_train==y_train).mean())
print('Accuracy of testing data: %.4f'%(y_hog_test==y_test).mean())

For HIST:
Accuracy of training data: 0.1358
Accuracy of testing data: 0.1444


For HOG:
Accuracy of training data: 0.2084
Accuracy of testing data: 0.2079


In [9]:
import numpy as np

def getAccuracy(y_true, y_pred):
    size = y_test.shape[0]
    return np.sum(y_true == y_pred) / size

print("hist train_acc:", getAccuracy(y_train, y_hist_train))
print("hist test_acc:", getAccuracy(y_test, y_hist_test))
print("hog train_acc:", getAccuracy(y_train, y_hog_train))
print("hog test_acc:", getAccuracy(y_test, y_hog_test))

hist train_acc: 0.6791
hist test_acc: 0.1444
hog train_acc: 1.0419
hog test_acc: 0.2079


In [10]:
print("-----Histogram result-----")
print("Training acc:", 100 * (y_hist_train == y_train).sum() / len(y_train), "%")
print("Testing acc:", 100 * (y_hist_test == y_test ).sum() / len(y_test), "%")

print("-----HOG result-----")
print("Training acc:", 100 * (y_hog_train == y_train).sum() / len(y_train), "%")
print("Testing acc:", 100 * (y_hog_test == y_test ).sum() / len(y_test), "%")

-----Histogram result-----
Training acc: 13.582 %
Testing acc: 14.44 %
-----HOG result-----
Training acc: 20.838 %
Testing acc: 20.79 %


如果你看到非常低的準確率，別懷疑，就是那麼低！到這裡大致可以體會到，靠人工設計的特徵在簡單的任務上也許是堪用（該例子如果用的是 mnist 數據集，準確應該還是可以有 90% 以上，有興趣的可以自己嘗試替換一下，或者參考[這個範例](https://github.com/opencv/opencv/blob/master/samples/python/tutorial_code/ml/py_svm_opencv/hogsvm.py)來跑看看），但複雜的情況，比如說分類的類別多起來，就能明顯感覺到這些特徵的不足之處了。

In [11]:
#!/usr/bin/env python

import cv2 as cv
import numpy as np

SZ=20
bin_n = 16 # Number of bins


affine_flags = cv.WARP_INVERSE_MAP|cv.INTER_LINEAR

## [deskew]
def deskew(img):
    m = cv.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11']/m['mu02']
    M = np.float32([[1, skew, -0.5*SZ*skew], [0, 1, 0]])
    img = cv.warpAffine(img,M,(SZ, SZ),flags=affine_flags)
    return img
## [deskew]

## [hog]
def hog(img):
    gx = cv.Sobel(img, cv.CV_32F, 1, 0)
    gy = cv.Sobel(img, cv.CV_32F, 0, 1)
    mag, ang = cv.cartToPolar(gx, gy)
    bins = np.int32(bin_n*ang/(2*np.pi))    # quantizing binvalues in (0...16)
    bin_cells = bins[:10,:10], bins[10:,:10], bins[:10,10:], bins[10:,10:]
    mag_cells = mag[:10,:10], mag[10:,:10], mag[:10,10:], mag[10:,10:]
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    hist = np.hstack(hists)     # hist is a 64 bit vector
    return hist
## [hog]

img = cv.imread('digits.png',0)
if img is None:
    raise Exception("we need the digits.png image from samples/data here !")


cells = [np.hsplit(row,100) for row in np.vsplit(img,50)]

# First half is trainData, remaining is testData
train_cells = [ i[:50] for i in cells ]
test_cells = [ i[50:] for i in cells]

######     Now training      ########################

deskewed = [list(map(deskew,row)) for row in train_cells]
hogdata = [list(map(hog,row)) for row in deskewed]
trainData = np.float32(hogdata).reshape(-1,64)
responses = np.repeat(np.arange(10),250)[:,np.newaxis]

svm = cv.ml.SVM_create()
svm.setKernel(cv.ml.SVM_LINEAR)
svm.setType(cv.ml.SVM_C_SVC)
svm.setC(2.67)
svm.setGamma(5.383)

svm.train(trainData, cv.ml.ROW_SAMPLE, responses)
svm.save('svm_data.dat')

######     Now testing      ########################

deskewed = [list(map(deskew,row)) for row in test_cells]
hogdata = [list(map(hog,row)) for row in deskewed]
testData = np.float32(hogdata).reshape(-1,bin_n*4)
result = svm.predict(testData)[1]

#######   Check Accuracy   ########################
mask = result==responses
correct = np.count_nonzero(mask)
print(correct*100.0/result.size)

Exception: we need the digits.png image from samples/data here !

## 參考資料


## 1. [Sobel 運算子 wiki](https://zh.wikipedia.org/wiki/%E7%B4%A2%E8%B2%9D%E7%88%BE%E7%AE%97%E5%AD%90)

## 2. [基於傳統圖像處理的目標檢測與識別(HOG+SVM附代碼)](https://www.cnblogs.com/zyly/p/9651261.html)

--

## 3. [知乎 - 什麼是 SVM](https://www.zhihu.com/question/21094489)

-

故事來源 : [Please explain Support Vector Machines (SVM) like I am a 5 year old](https://link.zhihu.com/?target=https%3A//www.reddit.com/r/MachineLearning/comments/15zrpp/please_explain_support_vector_machines_svm_like_i/)

-



### 故事是這樣子的：

在很久以前的情人節，大俠要去救他的愛人，但魔鬼和他玩了一個遊戲。

魔鬼在桌子上似乎有規律放了兩種顏色的球，說：“你用一根棍分開它們？要求：盡量在放更多球之後，仍然適用。”

![img1](https://pic1.zhimg.com/80/5aff2bcdbe23a8c764a32b1b5fb13b71_hd.jpg)

於是大俠這樣放，幹的不錯？ 

![img2](https://pic1.zhimg.com/80/3dbf3ba8f940dfcdaf877de2d590ddd1_hd.jpg)

然後魔鬼，又在桌上放了更多的球，似乎有一個球站錯了陣營。 

![img3](https://pic4.zhimg.com/80/0b2d0b26ec99ee40fd14760350e957af_hd.jpg)

SVM就是試圖把棍放在最佳位置，好讓在棍的兩邊有盡可能大的間隙。

![img4](https://pic4.zhimg.com/80/4b9e8a8a87c7982c548505574c13dc05_hd.jpg)


現在即使魔鬼放了更多的球，棍仍然是一個好的分界線。 

![img5](https://pic1.zhimg.com/80/7befaafc45763b9c4469abf245dc98cb_hd.jpg)


然後，在SVM工具箱中有另一個更加重要的trick。魔鬼看到大俠已經學會了一個trick，於是魔鬼給了大俠一個新的挑戰。 

![img6](https://pic2.zhimg.com/80/558161d10d1f0ffd2d7f9a46767de587_hd.jpg)


現在，大俠沒有棍可以很好幫他分開兩種球了，現在怎麼辦呢？當然像所有武俠片中一樣大俠桌子一拍，球飛到空中。然後，憑藉大俠的輕功，大俠抓起一張紙，插到了兩種球的中間。

![img7](https://pic3.zhimg.com/80/55d7ad2a6e23579b17aec0c3c9135eb3_hd.jpg)





現在，從魔鬼的角度看這些球，這些球看起來像是被一條曲線分開了。

![img8](https://pic2.zhimg.com/80/e5d5185561a4d5369f36a9737fc849c6_hd.jpg)


<font color="red">再之後，無聊的大人們，把這些球叫做「data」，把棍子叫做「classifier」 ,最大間隙trick叫做「optimization」，拍桌子叫做「kernelling」 ,那張紙叫做「hyperplane」。</font>





--

## 4. [程式碼範例的來源，裡面用的是 mnist 來跑 ](https://github.com/opencv/opencv/blob/master/samples/python/tutorial_code/ml/py_svm_opencv/hogsvm.py)
- [範例來源裡使用的 digit.png 檔案位置](https://raw.githubusercontent.com/opencv/opencv/master/samples/data/digits.png)